<a href="https://colab.research.google.com/github/0xf2/Pytorch-scratch/blob/master/Building_an_MNIST_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NOTE: Click the top-left "Open In Playground" button! 

In [ ]:
# install basical image libs
!pip install Pillow>=5.0.0
!pip install -U image

# install torch and torchvision (a utility library for computer vision that provides many public datasets and pre-trained models)
from os.path import exists
#from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
#platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.1.0-{platform}-linux_x86_64.whl torchvision

Requirement already up-to-date: image in /usr/local/lib/python3.6/dist-packages (1.5.33)


## Building an MNIST Classifier


Using what we have learned, let's build a simple MNIST digits classifier.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


cuda0 = torch.device('cuda:0')

### Loading Data

The [`torchvision`](https://pytorch.org/docs/stable/torchvision/index.html) library provides a wide range of standard vision datasets and networks with pretrained weights. We will use [the `torchvision.datasets.MNIST` class](https://pytorch.org/docs/stable/torchvision/datasets.html#mnist) to easily access the MNIST dataset.

In [ ]:
import torchvision

mnist_train = torchvision.datasets.MNIST(root='./data', download=True, train=True)

Each element of the dataset is a 2-tuple, consisting of the image of the digit, and its label. E.g.,

In [ ]:
image, label = mnist_train[13]
print('This is a digit {}:'.format(label))
image

This is a digit 6:


In [ ]:
type(image)

PIL.Image.Image

The image is given as a `PIL.Image`. To automatically obtain a `torch.Tensor`, we can add a [`torchvision.transforms.ToTensor` ](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.ToTensor) transform when constructing the dataset:

In [ ]:
mnist_train = torchvision.datasets.MNIST(root='./data', download=True, train=True,
                                         transform=torchvision.transforms.ToTensor())  # the ToTensor transform converts PIL.Image to torch.Tensor

In [ ]:
image, label = mnist_train[13]
print(type(image))
print(image.shape)  # MNIST images are 28x28, and have a single channel representing brightness.

<class 'torch.Tensor'>
torch.Size([1, 28, 28])


In [ ]:
print('This image has max={}'.format(image.max()), 'and min={}'.format(image.min()))

This image has max=1.0 and min=0.0


In deep learning, it is often a good idea to normalize network inputs to be centered around zero. We use the [`torchvision.transforms.Normalize`](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.Normalize) tranform to achieve this. Adding this transform, we construct the dataset as:

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,)),  # [0, 1] range => [-1, 1] range
])

mnist_train = torchvision.datasets.MNIST(root='./data', download=True, train=True,
                                         transform=transform)
mnist_val = torchvision.datasets.MNIST(root='./data', download=True, train=False,
                                         transform=transform)

In [ ]:
print('training set size:\t{}'.format(len(mnist_train)))
print('validation set size:\t{}'.format(len(mnist_val)))

training set size:	60000
validation set size:	10000


We use the PyTorch `torch.utils.data.DataLoader` to automatically load batched data. 

In [ ]:
batch_size = 512

train_loader = torch.utils.data.DataLoader(mnist_train, 
                                           batch_size=batch_size, 
                                           shuffle=True,                   # shuffle training set
                                           num_workers=4,                  # turns on multi-processing loading so training is not blocked by data loading
                                           pin_memory=True)                # pin_memory allows faster transfer from CPU to GPU
val_loader = torch.utils.data.DataLoader(mnist_val, 
                                         batch_size=batch_size, 
                                         num_workers=4, 
                                         pin_memory=True)

In [ ]:
# Each element yielded by `train_loader` (a Python iterable) is still a 2-tuple, 
# but now consisting of a batched image tensor, and a batched label tensor.

images, labels = next(iter(train_loader))
print('batched image tensor shape: {}'.format(images.shape))
print('batched label tensor shape: {}'.format(labels.shape))

batched image tensor shape: torch.Size([512, 1, 28, 28])
batched label tensor shape: torch.Size([512])


### Building the Network

We will use a convolutional network for classification. The following architecture is adapted from the famous [LeNet-5](https://ieeexplore.ieee.org/document/726791) [1].



[1] LeCun, Yann, et al. "Gradient-based learning applied to document recognition." Proceedings of the IEEE 86.11 (1998): 2278-2324.

In [ ]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = self.conv3(out)
        out = out.reshape(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [ ]:
net = MyNet().to(cuda0)

In [ ]:
# This network output a size 10 vector for each input image, as verified below 
# using a random input tensor.

net(torch.randn(32, 1, 28, 28, device=cuda0)).shape

torch.Size([32, 10])

### Training Loop


For classification, we will use the cross-entropy loss [`F.cross_entropy`](https://pytorch.org/docs/stable/nn.html#torch.nn.functional.cross_entropy) to train this network.

We can write a function that iterates through the training set (via `train_loader`) and train for 1 epoch. The code is extremely similar to what we did for approximating $f(x) = \cos(x)$, only with new ways to load data and compute loss.

In [ ]:
########################
#                      #
#       Exercise       #
#                      #
########################

# Fix the places with a `FIXME!!!`


def train(net, optim):
    net.train()
    for image, label in train_loader:
        # put data onto GPU
        # FIXME!!!
        
        # clear gradient
        # FIXME!!!
        
        # forward through the network
        # FIXME!!!
        
        # compute loss and gradient
        # FIXME!!! Check the `F.cross_entropy` documentation linked above
        
        # update parameters
        # FIXME!!!
        
        pass  # FIXME!!! Remove this `pass` when you are done.

Let's also write a function that evaluates our network on the validation set.

In [ ]:
########################
#                      #
#       Exercise       #
#                      #
########################

# Fix the places with a `FIXME!!!`


def evaluate(net):
    total = 0
    correct = 0
    
    net.eval()  # puts the network in eval mode. this is important when the 
                # network has layers that behaves differently in training and 
                # evaluation time, e.g., dropout and batch norm.
    for image, label in val_loader:
        # put data onto GPU
        # FIXME!!!
        
        with torch.no_grad():  # gradients are not tracked in this context manager
                               # since we are evaluating, gradients are not needed 
                               # and we can save some time and GPU memory.
              
            # forward through the network, and get the predicted class
            # FIXME!!!  (HINT: use .argmax(dim=-1))
            #   `prediction` should be an integer vector of size equal to the batch size.
            #   Remember that the network outputs logits of the prediction probabilities, 
            #   and that the higher the logits, the higher the probability.
            prediction = None  
            
            total += image.size(0)  # batch size
            correct += (prediction == label).sum().item()  # `.item()` retreives a python number from a 1-element tensor
            
    return correct / total

In [ ]:
# Without any training, the network accuracy matches that of random guessing: ~10%.

print('At initialization, the network has accuracy {:.4f}%'.format(evaluate(net) * 100))

At initialization, the network has accuracy 9.7300%


### Putting Everything Together

In [ ]:
########################
#                      #
#       Exercise       #
#                      #
########################

# Fix the places with a `FIXME!!!`


num_epochs = 10
lr = 0.1

optim = None  # FIXME!!! Construct an optimizer (e.g., SGD) to optimize the network parameters with the above learning rate

for epoch in range(num_epochs):
    print('Epoch: {}\tValidation Accuracy: {:.4f}%'.format(epoch, evaluate(net) * 100))
    train(net, optim)

print('Done! \tValidation Accuracy: {:.4f}%'.format(evaluate(net) * 100))

Epoch: 0	Validation Accuracy: 9.7300%
Epoch: 1	Validation Accuracy: 65.4300%
Epoch: 2	Validation Accuracy: 83.5600%
Epoch: 3	Validation Accuracy: 86.9800%
Epoch: 4	Validation Accuracy: 80.4700%
Epoch: 5	Validation Accuracy: 83.7900%
Epoch: 6	Validation Accuracy: 91.6700%
Epoch: 7	Validation Accuracy: 85.4100%
Epoch: 8	Validation Accuracy: 91.9300%
Epoch: 9	Validation Accuracy: 89.9600%
Done! 	Validation Accuracy: 91.8700%


## Open-ended Exercises (not required)

1. What could you do to make the accuracy higher and/or the training faster?  
2. Adapt the code for another dataset (e.g., CIFAR-10 from [`torchvision.datasets`](https://pytorch.org/docs/stable/torchvision/datasets.html)).
3. How dose your MNIST perform on [the USPS dataset](https://gist.github.com/SsnL/300865932b090d3f0798c2b961d371f9) (another black-white digits dataset)? Why?
4. How many samples are actually needed to train a good MNIST classifier? Try subsample the training set (sampled a fraction of images per class) and plot validation accuracy vs. training set size.
5. Use the [fast-gradient sign method](https://arxiv.org/abs/1412.6572) to attack your MNIST classifier! Why does your super great classifier now fail at a seemingly normal digit image?